In [8]:
# AiiDA環境をロード
from aiida import load_profile

# プロファイルをロード
load_profile("test_aiida")

Profile<uuid='efe3b646acbe4b46b34848999a0e7c25' name='test_aiida'>

In [9]:
# Int ノードをインポート
from aiida.orm import Int

node = Int(2)
node.store()

<Int: uuid: 1e3d13a8-5b83-42ba-b182-0bb010cf2052 (pk: 130) value: 2>

In [10]:
from aiida.engine import calcfunction


@calcfunction
def multiply(x, y):
    return x * y

In [11]:
x = Int(3)
y = Int(5)
multiply(x, y)

<Int: uuid: f4eaf863-2187-472e-9040-ed65509599a5 (pk: 134) value: 15>

In [16]:
from aiida.orm import load_code, load_node, load_group, Dict
from aiida.plugins import DataFactory

code = load_code('pw@localhost')
builder = code.get_builder()

structure = load_node(8)
builder.structure = structure

pseudo_family = load_group('SSSP/1.3/PBE/efficiency')
pseudos = pseudo_family.get_pseudos(structure=structure)
builder.pseudos = pseudos

parameters = {
  'CONTROL': {
    'calculation': 'scf',  # self-consistent field
  },
  'SYSTEM': {
    'ecutwfc': 30.,  # wave function cutoff in Ry
    'ecutrho': 240.,  # density cutoff in Ry
  },
}
builder.parameters = Dict(parameters)

KpointsData = DataFactory('core.array.kpoints')
kpoints = KpointsData()
kpoints.set_kpoints_mesh([4,4,4])
builder.kpoints = kpoints

builder.metadata.options.resources = {'num_machines': 1}
builder.metadata.options.queue_name = "GroupC"


In [17]:
from aiida.engine import submit
calcjob_node = submit(builder)